In this notebook, we will examine how to index and reindex a mongo database and the advantages of indexing the database

In [1]:
from pymongo import MongoClient, DESCENDING
import time
#DB connection
MONGODB_HOST = 'localhost'
MONGODB_PORT = 27017
DBS_NAME = 'KSCHOOL_TFM'
COLLECTION_NAME = 'tweets'
connection = MongoClient(MONGODB_HOST, MONGODB_PORT) #getting client
db_tweets = connection[DBS_NAME][COLLECTION_NAME] #getting db    

In [3]:
n_tweets = db_tweets.find().count()
print('There are {} tweets stored in the database'.format(n_tweets))

There are 1537251 tweets stored in the database


The time it takes to perform a query before indexing the tag field

In [2]:
%timeit db_tweets.find({'tag':'Jets'}).count()

1 loop, best of 3: 1min 20s per loop


Lets create an index on the field tag

In [7]:
db_tweets.create_index("tag")

u'tag_1'

The time it takes to perform a query after indexing the tag field

In [11]:
%timeit db_tweets.find({'tag':'Jets'}).count()

The slowest run took 4.19 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 181 µs per loop


How does it take to index the database? I'll examine this in another frequently queried field: datetime. Here however, I will specify indexing in DESCENDING order since we'll query mostly most frequent tweets.

In [40]:
t1 = time.time()
db_tweets.create_index([("dateTime", DESCENDING)])
t2 = time.time()
print("indexing the database takes {:.2f} seconds".format(t2-t1))

indexing the database takes 124.65 seconds


Lets index one more frequently queried field

In [42]:
db_tweets.create_index("lang")

u'lang_1'

Finally, since our database is constantly adding tweets, lets examine first how long it takes to query database after adding some unindexed fields and second how long it takes to reindex the whole database

In [44]:
n_tweets = db_tweets.find().count()
print('There are {} tweets stored in the database'.format(n_tweets))

There are 1539046 tweets stored in the database


In [46]:
print("There are {} new entries".format(1539046 -1537251))

There are 1795 new entries


Time in takes to query a field that did not update (no new tweet with this tag)

In [47]:
%timeit db_tweets.find({'tag':'Jets'}).count()

1000 loops, best of 3: 321 µs per loop


Time in takes to query a field that did update (new tweet with this tag)

In [48]:
%timeit db_tweets.find({'tag':'nuevo Gobierno'}).count()

1000 loops, best of 3: 521 µs per loop


Finally, how long does it take to reindex database?

In [50]:
t1 = time.time()
db_tweets.reindex()
t2 = time.time()
print("reindexing the database takes {:.2f} seconds".format(t2-t1))

reindexing the database takes 381.51 seconds
